In [27]:
import numpy as np
import scipy.io as sio
import scipy
import scipy.spatial
from scipy.linalg import norm
np.set_printoptions(threshold=100)

mat_contents = sio.loadmat('umist_cropped.mat')
facedat = mat_contents['facedat']

people = facedat[0]
people1 = facedat[0][0]
people2 = facedat[0][1]
people1_face1 = facedat[0][0][:,:,0]
people2_face2 = facedat[0][1][:,:,1]

vec_people1_face1 = people1_face1.flatten()
vec_people2_face2 = people2_face2.flatten()
pixels=112*92


In [28]:
people1_pic_size = facedat[0][0][:][91][1].size
print people1_pic_size
mat_people1 = np.zeros((people1_pic_size,pixels))
for i in range(0,people1_pic_size):
        mat_people1[i]= facedat[0][0][:,:,i].flatten()# mat_people1 是描述 class1 的大矩阵
vec_people1 = mat_people1.flatten()
#for test
#print mat_people1[0].reshape(112,92)

people2_pic_size = facedat[0][1][:][91][1].size
print people2_pic_size
mat_people2 = np.zeros((people2_pic_size,pixels))
for i in range(0,people2_pic_size):
        mat_people2[i]= facedat[0][1][:,:,i].flatten()# mat_people1 是描述 class1 的大矩阵
vec_people2 = mat_people2.flatten()
#for test
#print mat_people2[0].reshape(112,92)

def cal_distance_mat_people12(mat_peoplei,mat_peoplej):
    sizei = mat_peoplei.shape[0]
    sizej = mat_peoplej.shape[0]
    print sizei,sizej
    size = sizei+sizej
    print size
    mat_dist = np.zeros((size,size),dtype='f') 
    mat_people12=np.row_stack((mat_peoplei,mat_peoplej))
    for i in range(0,size):
        for j in range(0,size):
            mat_dist[i][j]= scipy.spatial.distance.euclidean(mat_people12[i],mat_people12[j])
    return mat_dist # the output is the distance matrix of the people1 and people2

mat_dist=cal_distance_mat_people12(mat_people1,mat_people2)
print mat_dist
print mat_dist[0][35]


38
35
38 35
73
[[    0.          1712.1583252   3003.83666992 ...,  7742.68994141
   8096.61083984  8547.62402344]
 [ 1712.1583252      0.          2379.18066406 ...,  7789.65966797
   8114.29882812  8594.61621094]
 [ 3003.83666992  2379.18066406     0.         ...,  7790.39892578
   8078.74072266  8658.35253906]
 ..., 
 [ 7742.68994141  7789.65966797  7790.39892578 ...,     0.          4006.83203125
   5862.74365234]
 [ 8096.61083984  8114.29882812  8078.74072266 ...,  4006.83203125     0.
   4748.14306641]
 [ 8547.62402344  8594.61621094  8658.35253906 ...,  5862.74365234
   4748.14306641     0.        ]]
9177.34


In [29]:
#计算class1的类内距离矩阵 ？ 封成函数？ 

def cal_mat_class_distance(mat_peoplei):  #（描述第i个class的矩阵 size*10340） 计算类内样本距离，类内样本距离向量
    rows = mat_peoplei.shape[0]
    dist = 0
    num = 0
    vec_class_distance = np.zeros((rows*(rows-1)/2),dtype='f')
    for i in range(0,rows-1):
        for j in range(i+1,rows):
            vec_class_distance[num] = scipy.spatial.distance.euclidean(mat_peoplei[i],mat_peoplei[j])
            dist += scipy.spatial.distance.euclidean(mat_peoplei[i],mat_peoplei[j])
            num += 1          
    return dist,vec_class_distance


In [30]:
print cal_mat_class_distance(mat_people1)

(4436164.7122043157, array([ 1712.1583252 ,  3003.83666992,  3113.13769531, ...,  6341.18847656,
        5616.08984375,  6287.3046875 ], dtype=float32))


In [31]:
def cal_av_mat_class_distance (mat_people): #计算类内样本平均距离
    dist = 0
    num = 0
    for item in mat_people:
        people_item_pic_size = item[:][91][1].size
        num += people_item_pic_size * (people_item_pic_size-1)/2
        # mat_people_item是描述 第item个class 的大矩阵
        mat_people_item = item.flatten().reshape(people_item_pic_size,112*92)
        vec_people_item = item.flatten()
        result_dist,result_vetor =  cal_mat_class_distance(mat_people_item)
        dist += result_dist
    av_dist = dist / num
    return av_dist

av_mat_class_distance = cal_av_mat_class_distance(people) # 平均类内样本距离
print av_mat_class_distance

1031.7384596


In [32]:
#计算class1 （people1）的类内距离标准差
dist1,vec_class_distance_people1 = cal_mat_class_distance(mat_people1) 
std1 = np.std(vec_class_distance_people1)
#计算class2 （people2）的类内距离标准差
dist2,vec_class_distance_people2 = cal_mat_class_distance(mat_people2) 
std2 = np.std(vec_class_distance_people2)
std12 = (std1+std2)/2
print std12


2111.22021484


In [33]:
mat_W = np.exp(-mat_dist/std12) #mat_W是 similarity matrix  核函数为 exp(-d/std)
print mat_W
print mat_W.shape

[[ 1.          0.44442213  0.24103901 ...,  0.02554279  0.02160051
   0.01744564]
 [ 0.44442213  1.          0.32402903 ...,  0.02498079  0.02142029
   0.01706162]
 [ 0.24103901  0.32402903  1.         ...,  0.02497205  0.02178412
   0.01655424]
 ..., 
 [ 0.02554279  0.02498079  0.02497205 ...,  1.          0.1498868
   0.06222829]
 [ 0.02160051  0.02142029  0.02178412 ...,  0.1498868   1.          0.10550424]
 [ 0.01744564  0.01706162  0.01655424 ...,  0.06222829  0.10550424  1.        ]]
(73, 73)


In [35]:
import python_ncut_lib as nc
#np.set_printoptions(threshehold = np.nan)
nbEigen = 3
eigen_value,vector=nc.ncut(mat_W,nbEigen)
vec_dis = nc.discretisation(vector)
print eigen_value
print vec_dis

[ 0.92952561  0.68883961  0.56011438]
  (15, 0)	1.0
  (16, 0)	1.0
  (17, 0)	1.0
  (18, 0)	1.0
  (19, 0)	1.0
  (20, 0)	1.0
  (21, 0)	1.0
  (22, 0)	1.0
  (23, 0)	1.0
  (48, 0)	1.0
  (49, 0)	1.0
  (50, 0)	1.0
  (51, 0)	1.0
  (52, 0)	1.0
  (53, 0)	1.0
  (54, 0)	1.0
  (55, 0)	1.0
  (56, 0)	1.0
  (57, 0)	1.0
  (58, 0)	1.0
  (59, 0)	1.0
  (60, 0)	1.0
  (61, 0)	1.0
  (62, 0)	1.0
  (66, 0)	1.0
  :	:
  (3, 2)	1.0
  (4, 2)	1.0
  (5, 2)	1.0
  (6, 2)	1.0
  (7, 2)	1.0
  (8, 2)	1.0
  (9, 2)	1.0
  (10, 2)	1.0
  (11, 2)	1.0
  (12, 2)	1.0
  (13, 2)	1.0
  (14, 2)	1.0
  (38, 2)	1.0
  (39, 2)	1.0
  (40, 2)	1.0
  (41, 2)	1.0
  (42, 2)	1.0
  (43, 2)	1.0
  (44, 2)	1.0
  (45, 2)	1.0
  (46, 2)	1.0
  (47, 2)	1.0
  (63, 2)	1.0
  (64, 2)	1.0
  (65, 2)	1.0
